In [15]:
from sklearn.model_selection import train_test_split
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.autograd import Variable


In [16]:
f = open("data/input.txt", "r")
text = f.read()
data_size = len(text)

dictionary = []
for i in text:
    if i not in dictionary:
        dictionary.append(i)
#print(dictionary)

chunk_len = 100
print()
dataset = []
k = 0
for char in text:
    ind = [dictionary.index(char)]
    dataset.append(ind)

for i in range(math.floor(len(text)/chunk_len)):
    word = []
    for j in range(k,k+chunk_len):
        word.append(text[j])
    k += chunk_len
    dataset.append(word)
dataset = text.split()


In [17]:
print(dictionary)
print(dataset[0:10])


['F', 'i', 'r', 's', 't', ' ', 'C', 'z', 'e', 'n', ':', '\n', 'B', 'f', 'o', 'w', 'p', 'c', 'd', 'a', 'y', 'u', 'h', ',', 'm', 'k', '.', 'A', 'l', 'S', 'Y', 'v', '?', 'R', 'M', 'W', "'", 'L', 'I', 'N', 'g', ';', 'b', '!', 'O', 'j', 'V', '-', 'T', 'H', 'E', 'U', 'D', 'P', 'q', 'x', 'J', 'G', 'K', 'Q', '&', 'Z', 'X', '3', '$']
['First', 'Citizen:', 'Before', 'we', 'proceed', 'any', 'further,', 'hear', 'me', 'speak.']


In [18]:
hidden_size = 200   # size of hidden state
seq_len = 100       # length of LSTM sequence
num_layers = 3      # num of layers in LSTM layer stack
learning_rate = 0.001          # learning rate
weight_decay = 1e-5            #weight decay
epochs = 20        # max number of epochs
drop_prob = 0.5
vocab_size = len(dictionary)  #length pf dictionary

In [19]:
class CharRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, drop_prob):
        super(CharRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, input_size)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        #self.drop_out = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, x, hidden_state):
        embedds = self.embedding(x)
        out, hidden_state = self.lstm(embedds, hidden_state)
        #out = self.drop_out(out)
        out = self.fc(out)
        return out, (hidden_state[0].detach(), hidden_state[1].detach())
    
    def initHidden(self, batch_size):
        return (Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size)))

In [ ]:
charRnn = CharRNN(vocab_size, vocab_size, hidden_size, num_layers, drop_prob)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(charRnn.parameters(), lr=learning_rate, weight_decay=weight_decay)